# Formality Classifier
This is going to be used to classify whether a sentence should be included in the informal or formal corpus. This will work by selecting the probability of the sentence belonging to the corpus, and if the score exceeds a threshold it will be included. 

In [2]:
import numpy as np
import seaborn as sns
import tensorflow as tf

import re 
import os
import pickle
import json
from datetime import datetime

from nltk.translate.bleu_score import sentence_bleu

from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

### Static Variables

In [3]:
BATCH_SIZE = 64
EMBEDDING_DIM = 200

## Load Data

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
# BASE_PATH = '../../Data'  # on local is path to directory
BASE_PATH = '/content/drive/MyDrive/Data/Data'

FORMAL_PATH_TRAIN = '{}/Supervised Data/Family_Relationships/S_Formal_FR_train.txt'.format(BASE_PATH)
INFORMAL_PATH_TRAIN = '{}/Supervised Data/Family_Relationships/S_Informal_FR_train.txt'.format(BASE_PATH)

FORMAL_PATH_HOLDOUT = '{}/Supervised Data/Family_Relationships/S_Formal_FR_ValTest.txt'.format(BASE_PATH)
INFORMAL_PATH_HOLDOUT = '{}/Supervised Data/Family_Relationships/S_Informal_FR_ValTest.txt'.format(BASE_PATH)

EMBEDDING_PATH = '{}/glove.6B.200d.txt'.format(BASE_PATH)

In [6]:
formal = open(FORMAL_PATH_TRAIN).read()
informal = open(INFORMAL_PATH_TRAIN).read()

formal_holdout = open(FORMAL_PATH_HOLDOUT).read()
informal_holdout = open(INFORMAL_PATH_HOLDOUT).read()

### Preprocess data

In [7]:
def process_sequence(seq):
    """This inserts a space in between the last word and a period"""
    s = re.sub('([.,!?()])', r' \1 ', seq)
    s = re.sub('\s{2,}', ' ', s)
    
    return '<start> ' + s + ' <end>'

In [8]:
f_corpus = [process_sequence(seq) for seq in formal.split('\n')]
if_corpus = [process_sequence(seq) for seq in informal.split('\n')]

f_holdout = [process_sequence(seq) for seq in formal_holdout.split('\n')]
if_holdout = [process_sequence(seq) for seq in informal_holdout.split('\n')]

print("Length of holdout set raw is", len(f_holdout))

f_val = f_holdout[:968]
if_val = if_holdout[:968]

if_holdout = if_holdout[968:]
f_holdout = f_holdout[968:]

Length of holdout set raw is 1968


In [9]:
def split_corpora(formal, informal):
    corpus = formal.copy()
    corpus.extend(informal)

    corpus_labels = [True for _ in range(len(formal))]
    corpus_labels.extend([False for _ in range(len(informal))])

    return corpus, corpus_labels

In [10]:
input_corpus, input_labels = split_corpora(f_corpus, if_corpus)
holdout_corpus, holdout_labels = split_corpora(f_holdout, if_holdout)
val_corpus, val_labels = split_corpora(f_val, if_val)

### Tokenize

In [11]:
def tokenize(corpus, tokenizer=None, maxlen=None):
    """ Tokenize data and pad sequences """
    if not tokenizer: 
        tokenizer = Tokenizer(filters='!"#$%&()*+,-./:;=?@[\\]^_`{|}~\t\n', 
                              oov_token='<OOV>')
        tokenizer.fit_on_texts(corpus)
    
    seqs = tokenizer.texts_to_sequences(corpus)
    padded_seqs = pad_sequences(seqs, padding='post', maxlen=maxlen)

    return padded_seqs, tokenizer

In [12]:
train_set, tokenizer = tokenize(input_corpus)
val_set, _ = tokenize(val_corpus, tokenizer)
test_set, _ = tokenize(holdout_corpus, tokenizer)

### Setup TF dataset

In [13]:
buffer_size = len(train_set)
steps_per_epoch = len(train_set) // BATCH_SIZE
vocab_size = len(tokenizer.word_index) + 1

train = tf.data.Dataset.from_tensor_slices((train_set, input_labels)).shuffle(buffer_size)
train = train.batch(BATCH_SIZE, drop_remainder=True)

test = tf.data.Dataset.from_tensor_slices((test_set, holdout_labels))
test = test.batch(BATCH_SIZE)

val = tf.data.Dataset.from_tensor_slices((val_set, val_labels))
val = val.batch(BATCH_SIZE)

In [14]:
example_input_batch, example_target_batch = next(iter(train))

### Load Embedding Weights

In [15]:
def embedding_matrix(tokenizer, vocab_size, embedding_dim):
    embeddings_index = {}
    with open(EMBEDDING_PATH) as f:
        for line in f:
            values = line.split()
            word = values[0]
            coefs = np.asarray(values[1:], dtype='float32')
            embeddings_index[word] = coefs

    embeddings_matrix = np.zeros((vocab_size, embedding_dim))
    for word, i in tokenizer.word_index.items():
        embedding_vector = embeddings_index.get(word)
        if embedding_vector is not None:
            embeddings_matrix[i] = embedding_vector

    return embeddings_matrix

In [16]:
E = embedding_matrix(tokenizer, vocab_size, EMBEDDING_DIM)

## Declare Model

In [30]:
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, EMBEDDING_DIM, weights=[E], mask_zero=True), 
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.LSTM(512),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(128, activation="relu"),
    tf.keras.layers.Dense(2, activation='softmax')
])

In [31]:
model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [32]:
history = model.fit(train, validation_data=val, epochs=5)

Epoch 1/5
 37/781 [>.............................] - ETA: 9:13 - loss: 0.6349 - accuracy: 0.6220

KeyboardInterrupt: ignored

In [ ]:
model.evaluate(test)

Going to find the following data set 
$$ T_{avg} = \{(s_i, s_i^\prime)|P_+(s_i^\prime) - P_+(s_i) > \sigma \}$$

In [ ]:
model.predict(test)

array([[0.49983445, 0.5001656 ],
       [0.4999894 , 0.5000106 ],
       [0.5001057 , 0.49989432],
       ...,
       [0.500086  , 0.49991402],
       [0.5000835 , 0.49991652],
       [0.50109226, 0.49890772]], dtype=float32)